In [11]:
# Config
import os

# Basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Monitoring
from tqdm.notebook import tqdm

# IO
from os.path import join, exists, basename, dirname, splitext, expanduser
from glob import glob

# Parallel processing
from joblib import Parallel, delayed
import re
from PIL import Image
import supervision as sv
from supervision.metrics import MeanAveragePrecision
from supervision.metrics import MetricTarget
from tempfile import mkdtemp

from ultralytics import YOLO


In [12]:
#mention cuda device
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
home_dir="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/"
base_state = "haryana_normal" ## state for which model is trained
target_state = "bihar_genarated" ## state for which model is to be validated
test_state = "test_bihar" ## state for which model is to be validatd during training
model_name = "yolo11m-obb.pt" ## trained model name
task="obb" ## task for which model is trained
conf=0.001 ## confidence threshold
iou=0.33 ## iou threshold
imagsz=2560 ## image size
epochs=100 ## number of epochs
# ratio="same_class_count_10_120_1000_4x"

# base_dir="/home/patel_zeel/kiln_compass_24/data/"

gt_image_dir= f"/home/umang.shikarvar/experiment/haryana_like_bihar/images"
gt_label_dir=f"/home/umang.shikarvar/experiment/haryana_like_bihar/labels"
# gt_image_dir= "/home/patel_zeel/kiln_compass_24/data/m0_obb_without_empty_val/images"
# gt_label_dir="/home/patel_zeel/kiln_compass_24/data/m0_obb_without_empty_val/labels"
print(gt_image_dir)
print(gt_label_dir)

## class information in yml file 
data_yml_save_path=mkdtemp()
data_yml = f"""train: dummy
val: dummy
nc: 3
names: ["CFCBK", "FCBK", "Zigzag"]
"""
data_yml_path = f"{data_yml_save_path}/data.yml"
with open(data_yml_path, "w") as f:
    f.write(data_yml)

print(data_yml_path)  


class_count="80_20"
type="obb"
## model path
yolo_runs_dir = f"/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/{type}/labels/"
model_path = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_haryana_test_bihar_same_class_count_obb_v1_yolo11m-obb.pt_640_128_300/weights/best.pt"
# output path
result_dir=f"{yolo_runs_dir}train_{base_state}_test_{target_state}_{class_count}_{task}_v2_{model_name}_{imagsz}_128_{epochs}_{conf}_{iou}/"
prediction_dir = join(result_dir,"labels")
print(result_dir)
print(prediction_dir)


/home/umang.shikarvar/experiment/haryana_like_bihar/images
/home/umang.shikarvar/experiment/haryana_like_bihar/labels
/tmp/tmpag3xj0kp/data.yml
/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_haryana_normal_test_bihar_genarated_80_20_obb_v2_yolo11m-obb.pt_2560_128_100_0.001_0.33/
/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_haryana_normal_test_bihar_genarated_80_20_obb_v2_yolo11m-obb.pt_2560_128_100_0.001_0.33/labels


In [13]:
## Supervision dataset
sv_dataset=sv.DetectionDataset.from_yolo(gt_image_dir,gt_label_dir,data_yml_path)
print(sv_dataset)
print(len(sv_dataset))
sv_dataset.classes

## Yolo model
model = YOLO(model_path)

687


In [14]:
targets=[]
predictions=[]
for name,_,gt_detection in tqdm(sv_dataset):
    file_name=splitext(basename(name))[0]
    # prediction_path=join(prediction_dir,f"{file_name}.txt")
    ultralytics_result=model(name,imgsz=640,iou=0.33,conf=0.001,exist_ok=True,save_txt=True,max_det=300,name=result_dir, verbose=False)[0]
    sv_detection=sv.Detections.from_ultralytics(ultralytics_result)
    targets.append(gt_detection)
    predictions.append(sv_detection)

  0%|          | 0/687 [00:00<?, ?it/s]

Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_haryana_normal_test_bihar_genarated_80_20_obb_v2_yolo11m-obb.pt_2560_128_100_0.001_0.33
674 labels saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_haryana_normal_test_bihar_genarated_80_20_obb_v2_yolo11m-obb.pt_2560_128_100_0.001_0.33/labels
Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_haryana_normal_test_bihar_genarated_80_20_obb_v2_yolo11m-obb.pt_2560_128_100_0.001_0.33
674 labels saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_haryana_normal_test_bihar_genarated_80_20_obb_v2_yolo11m-obb.pt_2560_128_100_0.001_0.33/labels
Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_haryana_normal_test_bihar_genarated_80_20_obb_v2_yolo11m-obb.pt_2560_128_100_0.001_0.33
674 labels saved to /home/rishabh.mondal/Brick-Kilns-pr

In [17]:
## mAP calculation 
mAP_metric = MeanAveragePrecision(class_agnostic=False)

mAP_result=mAP_metric.update(predictions,targets).compute()
class_wise_mAP=mAP_result.ap_per_class[:,0].tolist()
# print(f"mAP_result: {mAP_result}")
matched_classes=mAP_result.matched_classes.tolist()
print(f"Matched classes: {matched_classes}")
num_classes=3
final_class_wise_mAP = [0]*num_classes

for cls, mAP in zip(matched_classes, class_wise_mAP):
    print(f"cls: {cls}, mAP: {mAP}")
    final_class_wise_mAP[cls] = mAP


print(f"class_wise_mAP: {final_class_wise_mAP}")


mAP_metric = MeanAveragePrecision(class_agnostic=True)
mAP_result = mAP_metric.update(predictions, targets).compute()
class_agnostic_result = mAP_result.ap_per_class[:, 0].tolist()
# print(f"mAP_result: {mAP_result}")
print(f"class_wise_mAP: {class_wise_mAP}")
print(f"class_agnostic_result: {class_agnostic_result}")

Matched classes: [0, 1, 2]
cls: 0, mAP: 0.0
cls: 1, mAP: 0.0
cls: 2, mAP: 0.16017220033293583
class_wise_mAP: [0.0, 0.0, 0.16017220033293583]
class_wise_mAP: [0.0, 0.0, 0.16017220033293583]
class_agnostic_result: [0.1518088352140639]


In [16]:
target_state = "test_bihar_swir_2560"
base_state = "train_bihar_swir_2560" 

In [13]:
index = pd.MultiIndex.from_tuples([], names=["Base State", "class_count", "Target State"])
result_df = pd.DataFrame(columns=["CFCBK", "FCBK", "Zigzag", "mAP", "Class-agnostic AP"], index=index)
result_df.loc[(base_state, class_count, target_state), :] = final_class_wise_mAP + [mAP_result.mAP_scores[0]] + class_agnostic_result
display(result_df)


,,,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
Base State,class_count,Target State,,,,,
train_bihar_swir_2560,80_20,test_bihar_swir_2560,0.44769,0.010765,0.583639,0.556565,0.556565


In [14]:
output_csv_path = f"{home_dir}result_csv/different_model_performance_m0.csv"


In [15]:
# output_csv_path = f"{home_dir}result_csv/different_state_to_{target_state}_mAP_results.csv"
os.makedirs(dirname(output_csv_path), exist_ok=True)


if not os.path.exists(output_csv_path):
    result_df.to_csv(output_csv_path, header=True)
    print(f"Results saved to {output_csv_path}")
else:
    result_df.to_csv(output_csv_path, mode='a', header=False)
    print(f"Results appended to {output_csv_path}")

Results appended to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/result_csv/different_model_performance_m0.csv


In [17]:
df="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/result_csv/different_model_performance_m0.csv"
df = pd.read_csv(df)
df

,Base State,class_count,Target State,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
0,m0_obb_train_swinir_4x,80_20,m0_obb_val_swinir_4x,0.748231,0.630102,0.847745,0.876737,0.876737
1,m0_obb_train_640,80_20,m0_obb_val_640,0.676640,0.602387,0.809013,0.833650,0.833650
2,m0_obb_train_2560,80_20,m0_obb_val_2560,0.624660,0.618327,0.839918,0.838759,0.838759
3,train_bihar_640,80_20,test_bihar_640,0.000000,0.168050,0.606768,0.555936,0.555936
4,train_haryana_swir_2560,80_20,test_bihar_swir_2560,0.082210,0.029586,0.462763,0.465030,0.465030
5,train_bihar_swir_2560,80_20,test_bihar_swir_2560,0.568347,0.014577,0.594423,0.595148,0.595148


In [18]:
display(df)

,Base State,class_count,Target State,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
0,m0_obb_train_swinir_4x,80_20,m0_obb_val_swinir_4x,0.748231,0.630102,0.847745,0.876737,0.876737
1,m0_obb_train_640,80_20,m0_obb_val_640,0.676640,0.602387,0.809013,0.833650,0.833650
2,m0_obb_train_2560,80_20,m0_obb_val_2560,0.624660,0.618327,0.839918,0.838759,0.838759
3,train_bihar_640,80_20,test_bihar_640,0.000000,0.168050,0.606768,0.555936,0.555936
4,train_haryana_swir_2560,80_20,test_bihar_swir_2560,0.082210,0.029586,0.462763,0.465030,0.465030
5,train_bihar_swir_2560,80_20,test_bihar_swir_2560,0.568347,0.014577,0.594423,0.595148,0.595148


In [5]:
base_state_2 =  ## state for which model is trained
output_csv_path_2 = f"{home_dir}result_csv/{base_state_2}_to_different_state_mAP_results.csv"

result_west_bengal=pd.read_csv(output_csv_path)
result_punjab=pd.read_csv(output_csv_path_2)
result=pd.concat([result_west_bengal,result_punjab],ignore_index=True)
display(result)

SyntaxError: invalid syntax (1003467514.py, line 1)

In [6]:
csv_path=f"../result_csv/different_state_to_bihar_same_class_count_10_120_1000_v4_mAP_results.csv"
result=pd.read_csv(csv_path)
# sort by mAP
result=result.sort_values(by='mAP',ascending=False)
display(result.style.background_gradient(cmap='viridis',axis=0))

NameError: name 'pd' is not defined

In [29]:
def apply_state_color(val):
    if val == 'West Bengal':
        return 'background-color: yellow'  # color for West Bengal (Yellow)
    elif val == 'Punjab':
        return 'background-color: green'  # color for Punjab (Green)
    else:
        return ''  # no color if it's other

# Apply the color styling for 'Base State' and 'Target State' columns
styled_result = result.style\
    .applymap(apply_state_color, subset=["Base State", "Target State"])\
    .background_gradient(subset=["CFCBK", "FCBK", "Zigzag", "mAP", "Class-agnostic AP"], cmap="coolwarm")\
    .set_properties(**{"text-align": "center"})\
    .set_table_styles([
        {"selector": "th", "props": [("font-weight", "bold"), ("text-align", "center")]},
        {"selector": "tr:nth-child(even)", "props": [("background-color", "#f9f9f9")]}
    ])

# Displaying the styled DataFrame
display(styled_result)

,Base State,Target State,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
0,west_bengal,punjab,0.000000,0.019683,0.425855,0.492828,0.492828
1,west_bengal,uttar_pradesh,0.000000,0.129238,0.197902,0.417703,0.417703
2,west_bengal,haryana,0.000000,0.004189,0.236194,0.270739,0.270739
3,west_bengal,bihar,0.000000,0.033873,0.412915,0.451084,0.451084
4,punjab,haryana,0.000000,0.008151,0.643177,0.659907,0.659907
5,punjab,bihar,0.000000,0.012281,0.338602,0.322256,0.322256
6,punjab,uttar_pradesh,0.000000,0.035343,0.137244,0.215012,0.215012
7,punjab,west_bengal,0.000000,0.016273,0.133746,0.199013,0.199013
